In [1]:
from typing import ClassVar

from gemd import (
    Parameter,
    Property, 
    PropertyAndConditions,
    MaterialRun,
    PropertyTemplate,
    ProcessTemplate,
    MaterialTemplate,
    ParameterTemplate,
    CategoricalBounds,
    NominalCategorical,
    RealBounds,
    NominalReal,
)

from openmsimodel.entity.gemd.process import Process
from openmsimodel.entity.gemd.material import Material
from openmsimodel.entity.gemd.gemd_element import GEMDElement

from openmsimodel.utilities.attributes import (
    AttrsDict,
    define_attribute,
    finalize_template,
)

### Subclassing  

One initialization method allows you to subclass a GEMDElement object to initialize its template and attributes, and preserve it as a persistent class in a file

In [2]:
class Homogenization(Process):
    """Class representing homogenization cycle"""

    TEMPLATE: ClassVar[ProcessTemplate] = ProcessTemplate(
        name="Homogenization",
        description="""Homogenizing
                """,
    )

    _ATTRS: ClassVar[AttrsDict] = {"conditions": {}, "parameters": {}}

    define_attribute(
        _ATTRS,
        template=ParameterTemplate(
            name="Pressure",
            bounds=RealBounds(0, 20, "Pa"),
        ),
    )

    define_attribute(
        _ATTRS,
        template=ParameterTemplate(
            name="Duration",
            bounds=RealBounds(0, 72, "hours"),
        ),
    )

    define_attribute(
        _ATTRS,
        template=ParameterTemplate(
            name="Temperature",
            bounds=RealBounds(0, 2000, "Kelvin"),
        ),
    )

    finalize_template(_ATTRS, TEMPLATE)

homogenization_process = Homogenization('Homogenization process')
print(homogenization_process._ATTRS)


{'conditions': {}, 'parameters': {'Pressure': {'obj': {'description': None, 'bounds': {'lower_bound': 0, 'upper_bound': 20, 'default_units': 'pascal', 'type': 'real_bounds'}, 'name': 'Pressure', 'tags': [], 'uids': {'auto': '7befc409-880f-48e2-8bce-3b06d9a577ef'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Duration': {'obj': {'description': None, 'bounds': {'lower_bound': 0, 'upper_bound': 72, 'default_units': 'hour', 'type': 'real_bounds'}, 'name': 'Duration', 'tags': [], 'uids': {'auto': 'c2b0f86d-af3b-497a-aac7-df2ff75a79eb'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Temperature': {'obj': {'description': None, 'bounds': {'lower_bound': 0, 'upper_bound': 2000, 'default_units': 'kelvin', 'type': 'real_bounds'}, 'name': 'Temperature', 'tags': [], 'uids': {'auto': '00d8b1fc-7e4c-417a-b858-897fde8f94a3'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}}}


In [3]:
homogenization_process.assert_linked()

template and spec are linked properly.
run and spec are linked properly.


Attempting to initialize without a template will lead to an error

In [4]:
try:
    homogenization_process = Process('Homogenization process')
except AttributeError as e:
    print(e)

TEMPLATE is not defined. Assign 'template' parameter, or create a new subclass with a defined TEMPLATE attribute.


### Manipulating Attributes

The attributes API functions are two, one for updating and the other for removing. They are applicable to any descendent of GEMDElement, so all 4 of Process, Material, Ingredient and Measurement inherit these methods.

In [5]:
homogenization_process._update_attributes(
    AttrType=Parameter,
    attributes=(Parameter(
        "Pressure",
        value=NominalReal(10.0, "Pa"),
        template=homogenization_process._ATTRS["parameters"][
            "Pressure"]["obj"],
        origin="specified"
        ), ),
    which='run'
)
print(homogenization_process.run.parameters)

[{'file_links': [], 'origin': 'specified', 'name': 'Pressure', 'value': {'units': 'pascal', 'nominal': 10.0, 'type': 'nominal_real'}, 'template': {'description': None, 'bounds': {'lower_bound': 0, 'upper_bound': 20, 'default_units': 'pascal', 'type': 'real_bounds'}, 'name': 'Pressure', 'tags': [], 'uids': {'auto': '7befc409-880f-48e2-8bce-3b06d9a577ef'}, 'type': 'parameter_template'}, 'notes': None, 'type': 'parameter'}]


In [6]:
homogenization_process._remove_attributes(Parameter, attr_names=("Pressure",), which='run')
print(homogenization_process.run.parameters)

[]


### Manipulating attributes more granularly

In [7]:
metal_material = Material('Metal', template=MaterialTemplate('Metal')) #properties=PropertyTemplate('Conductivity')))
test = MaterialRun('test')
metal_material.update_properties(
    (
        PropertyAndConditions(
            property=Property('Conductivity', 
                value=NominalReal(5, "s/m"),
                template=PropertyTemplate('Conductivity', bounds=RealBounds(0, 20, "s/m")), 
            ), 
            conditions=[])
    ),
    which='spec'
)
print(metal_material.spec.properties)

[{'conditions': [], 'property': {'file_links': [], 'origin': 'unknown', 'name': 'Conductivity', 'value': {'units': 'second / meter', 'nominal': 5.0, 'type': 'nominal_real'}, 'template': {'description': None, 'bounds': {'lower_bound': 0, 'upper_bound': 20, 'default_units': 'second / meter', 'type': 'real_bounds'}, 'name': 'Conductivity', 'tags': [], 'uids': {}, 'type': 'property_template'}, 'notes': None, 'type': 'property'}, 'type': 'property_and_conditions'}]


In [8]:
metal_material.remove_properties('Conductivity')
print(metal_material.spec.properties)

[]


### Manipulating other assets, such as tags and file links

In [16]:
metal_material.update_tags(('Name', 'Fe'), which='run')
print(metal_material.run.tags)

['Name::Fe']


In [18]:
metal_material.remove_tags(('Name', 'Fe'), which='run')
print(metal_material.run.tags)

[]
